# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fe62432aac0>
├── 'a' --> tensor([[ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493]])
└── 'x' --> <FastTreeValue 0x7fe62740d760>
    └── 'c' --> tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                        [-0.7616,  0.7960, -0.3887, -0.2607],
                        [-0.0434,  0.9229,  1.4960,  0.8869]])

In [4]:
t.a

tensor([[ 1.1171, -0.4977,  0.5248],
        [ 1.9999,  1.1367,  0.9493]])

In [5]:
%timeit t.a

63.5 ns ± 0.0851 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fe62432aac0>
├── 'a' --> tensor([[ 0.3425, -0.4352, -0.4775],
│                   [-0.2628,  0.7990, -0.4734]])
└── 'x' --> <FastTreeValue 0x7fe62740d760>
    └── 'c' --> tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                        [-0.7616,  0.7960, -0.3887, -0.2607],
                        [-0.0434,  0.9229,  1.4960,  0.8869]])

In [7]:
%timeit t.a = new_value

74.1 ns ± 0.0344 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 1.1171, -0.4977,  0.5248],
               [ 1.9999,  1.1367,  0.9493]]),
    x: Batch(
           c: tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869]]),
       ),
)

In [10]:
b.a

tensor([[ 1.1171, -0.4977,  0.5248],
        [ 1.9999,  1.1367,  0.9493]])

In [11]:
%timeit b.a

58.2 ns ± 0.167 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.1346, -1.4281,  0.7654],
               [-1.0446, -0.3884, -0.0966]]),
    x: Batch(
           c: tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869]]),
       ),
)

In [13]:
%timeit b.a = new_value

497 ns ± 0.0856 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

845 ns ± 1.94 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

10.7 µs ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

144 µs ± 231 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fe57ec5af70>
├── 'a' --> tensor([[[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]],
│           
│                   [[ 1.1171, -0.4977,  0.5248],
│                    [ 1.9999,  1.1367,  0.9493]]])
└── 'x' --> <FastTreeValue 0x7fe57ec5ae80>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.2 µs ± 55.3 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fe57ec80a90>
├── 'a' --> tensor([[ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493],
│                   [ 1.1171, -0.4977,  0.5248],
│                   [ 1.9999,  1.1367,  0.9493]])
└── 'x' --> <FastTreeValue 0x7fe57ec80b80>
    └── 'c' --> tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                        [-0.7616,  0.7960, -0.3887, -0.2607],
                 

In [23]:
%timeit t_cat(trees)

30.4 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

57.1 µs ± 69.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 1.6494, -0.5900, -0.2535,  0.1055],
                       [-0.7616,  0.7960, -0.3887, -0.2607],
                       [-0.0434,  0.9229,  1.4960,  0.8869]],
              
                      [[ 1.6494, -0.5900, -0.2535,  0.1055],
                       [-0.7616,  0.7960, -0.3887, -0.2607],
                       [-0.0434,  0.9229,  1.4960,  0.8869]],
              
                      [[ 1.6494, -0.5900, -0.2535,  0.1055],
                       [-0.7616,  0.7960, -0.3887, -0.2607],
                       [-0.0434,  0.9229,  1.4960,  0.8869]],
              
                      [[ 1.6494, -0.5900, -0.2535,  0.1055],
                       [-0.7616,  0.7960, -0.3887, -0.2607],
                       [-0.0434,  0.9229,  1.4960,  0.8869]],
              
                      [[ 1.6494, -0.5900, -0.2535,  0.1055],
                       [-0.7616,  0.7960, -0.3887, -0.2607],
                       [-0.0434,  0.9229,  1.4960,  0.8869]],

In [26]:
%timeit Batch.stack(batches)

76.5 µs ± 281 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869],
                      [ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869],
                      [ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869],
                      [ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869],
                      [ 1.6494, -0.5900, -0.2535,  0.1055],
                      [-0.7616,  0.7960, -0.3887, -0.2607],
                      [-0.0434,  0.9229,  1.4960,  0.8869],
                      [ 1.6494, -0.5900, -0.2535,  0.1055],
                   

In [28]:
%timeit Batch.cat(batches)

145 µs ± 133 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

329 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
